## Análise Fatorial e PCA
### MBA em Data Science e Analytics USP ESALQ

**Prof Dr.** Wilson Tarantin Junior

**Aluna:** Luiza Batista Laquini

**Turma:** DSA 2024-1

### Bibliotecas e configurações

In [ ]:
import pandas as pd
import numpy as np
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

#import plotly.io as pio
#pio.renderers.default='browser'

import warnings
warnings.filterwarnings("ignore")

### Visualização dos dados

In [ ]:
casas = pd.read_excel("preco_casas.xlsx")
# Fonte: adaptado de https://www.kaggle.com/datasets/elakiricoder/jiffs-house-price-prediction-dataset

Ã‰ possÃ­vel extrairmos fatores que capturem o preÃ§o de venda das casas?

% EstatÃ­sticas descritiva das variÃ¡veis

In [ ]:
tab_desc = casas.describe()

% Analisando as correlaÃ§Ãµes de Pearson

Matriz de correlaÃ§Ãµes

In [ ]:
corr = casas.corr()

GrÃ¡fico interativo

In [ ]:
fig = go.Figure()

In [ ]:
fig.add_trace(
    go.Heatmap(
        x = corr.columns,
        y = corr.index,
        z = np.array(corr),
        text=corr.values,
        texttemplate='%{text:.3f}',
        colorscale='viridis'))

In [ ]:
fig.update_layout(
    height = 750,
    width = 750,
    yaxis=dict(autorange="reversed"))

In [ ]:
fig.show()

% Selecionando as variÃ¡veis de interesse para a anÃ¡lise

Vamos deixar o preÃ§o das casas de fora da anÃ¡lise fatorial! 

In [ ]:
casas_pca = casas.drop(columns=['property_value'])

% Teste de Esfericidade de Bartlett

In [ ]:
bartlett, p_value = calculate_bartlett_sphericity(casas_pca)

In [ ]:
print(f'QuiÂ² Bartlett: {round(bartlett, 2)}')
print(f'p-valor: {round(p_value, 4)}')

% Definindo a PCA (procedimento inicial extraindo todos os fatores possÃ­veis)

In [ ]:
fa = FactorAnalyzer(n_factors=8, method='principal', rotation=None).fit(casas_pca)

% Obtendo os autovalores

In [ ]:
autovalores = fa.get_eigenvalues()[0]

In [ ]:
print(autovalores)

Soma dos autovalores

In [ ]:
round(autovalores.sum(), 2)

% CritÃ©rio de Kaiser (raiz latente)

Temos 3 autovalores maiores do que 1<br>
Vamos parametrizar a funÃ§Ã£o para a extraÃ§Ã£o de 3 fatores!

In [ ]:
fa = FactorAnalyzer(n_factors=3, method='principal', rotation=None).fit(casas_pca)

% Eigenvalues, variÃ¢ncias e variÃ¢ncias acumuladas

In [ ]:
autovalores_fatores = fa.get_factor_variance()

In [ ]:
tabela_eigen = pd.DataFrame(autovalores_fatores)
tabela_eigen.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_eigen.columns)]
tabela_eigen.index = ['Autovalor','VariÃ¢ncia', 'VariÃ¢ncia Acumulada']
tabela_eigen = tabela_eigen.T

In [ ]:
print(tabela_eigen)

% GrÃ¡fico da variÃ¢ncia acumulada dos componentes principais

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.barplot(x=tabela_eigen.index, y=tabela_eigen['VariÃ¢ncia'], data=tabela_eigen, palette='pastel')
ax.bar_label(ax.containers[0])
plt.title("Fatores ExtraÃ­dos", fontsize=16)
plt.xlabel(f"{tabela_eigen.shape[0]} fatores que explicam {round(tabela_eigen['VariÃ¢ncia'].sum()*100,2)}% da variÃ¢ncia", fontsize=12)
plt.ylabel("Porcentagem de variÃ¢ncia explicada", fontsize=12)
plt.show()

% Determinando as cargas fatoriais

In [ ]:
cargas_fatoriais = fa.loadings_

In [ ]:
tabela_cargas = pd.DataFrame(cargas_fatoriais)
tabela_cargas.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_cargas.columns)]
tabela_cargas.index = casas_pca.columns

In [ ]:
print(tabela_cargas)

% Analisando as cargas fatoriais em cada fator extraÃ­do

In [ ]:
tabela_cargas_graph = tabela_cargas.reset_index()
tabela_cargas_graph = tabela_cargas_graph.melt(id_vars='index')

In [ ]:
sns.barplot(data=tabela_cargas_graph, x='variable', y='value', hue='index', palette='bright')
plt.legend(title='VariÃ¡veis', bbox_to_anchor=(1,1), fontsize = '6')
plt.title('Cargas Fatoriais', fontsize='12')
plt.xlabel(xlabel=None)
plt.ylabel(ylabel=None)
plt.show()

% Determinando as comunalidades

In [ ]:
comunalidades = fa.get_communalities()

In [ ]:
tabela_comunalidades = pd.DataFrame(comunalidades)
tabela_comunalidades.columns = ['Comunalidades']
tabela_comunalidades.index = casas_pca.columns

In [ ]:
print(tabela_comunalidades)

% ExtraÃ§Ã£o dos fatores para as observaÃ§Ãµes do banco de dados

In [ ]:
fatores = pd.DataFrame(fa.transform(casas_pca))
fatores.columns =  [f"Fator {i+1}" for i, v in enumerate(fatores.columns)]

Adicionando os fatores ao banco de dados

In [ ]:
casas = pd.concat([casas.reset_index(drop=True), fatores], axis=1)

% Identificando os scores fatoriais

In [ ]:
scores = fa.weights_

In [ ]:
tabela_scores = pd.DataFrame(scores)
tabela_scores.columns = [f"Fator {i+1}" for i, v in enumerate(tabela_scores.columns)]
tabela_scores.index = casas_pca.columns

In [ ]:
print(tabela_scores)

% Analisando os scores fatoriais em cada fator extraÃ­do

In [ ]:
tabela_scores_graph = tabela_scores.reset_index()
tabela_scores_graph = tabela_scores_graph.melt(id_vars='index')

In [ ]:
sns.barplot(data=tabela_scores_graph, x='variable', y='value', hue='index', palette='rocket')
plt.legend(title='VariÃ¡veis', bbox_to_anchor=(1,1), fontsize = '6')
plt.title('Scores Fatoriais', fontsize='12')
plt.xlabel(xlabel=None)
plt.ylabel(ylabel=None)
plt.show()

% Vamos consolidar os 3 fatores em uma medida Ãºnica (soma ponderada)

In [ ]:
casas['Ranking'] = 0

In [ ]:
for index, item in enumerate(list(tabela_eigen.index)):
    variancia = tabela_eigen.loc[item]['VariÃ¢ncia']
    casas['Ranking'] = casas['Ranking'] + casas[tabela_eigen.index[index]]*variancia

% Os preÃ§os alinham-se Ã s caracterÃ­sticas representadas nos fatores?

In [ ]:
pg.rcorr(casas[['Ranking', 'property_value']], 
         method = 'pearson', upper = 'pval', 
         decimals = 4, 
         pval_stars = {0.01: '***', 0.05: '**', 0.10: '*'})

% Fim!